In [1]:
!pip install tensorflow==1.13.1

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import urllib
import os
import tensorflow as tf
import json
from zipfile import ZipFile

In [20]:
def process_bb_box(food_image_url):
    with open(food_image_url) as f:
        bb_json = json.load(f)
        return bb_json
    

In [21]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Utility functions for creating TFRecord data sets."""



def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def read_examples_list(path):
  """Read list of training or validation examples.
  The file is assumed to contain a single example per line where the first
  token in the line is an identifier that allows us to find the image and
  annotation xml for that example.
  For example, the line:
  xyz 3
  would allow us to find files xyz.jpg and xyz.xml (the 3 would be ignored).
  Args:
    path: absolute path to examples list file.
  Returns:
    list of example identifiers (strings).
  """
  with tf.gfile.GFile(path) as fid:
    lines = fid.readlines()
  return [line.strip().split(' ')[0] for line in lines]


def recursive_parse_xml_to_dict(xml):
  """Recursively parses XML contents to python dict.
  We assume that `object` tags are the only ones that can appear
  multiple times at the same level of a tree.
  Args:
    xml: xml tree obtained by parsing XML file contents using lxml.etree
  Returns:
    Python dictionary holding XML contents.
  """
  if not xml:
    return {xml.tag: xml.text}
  result = {}
  for child in xml:
    child_result = recursive_parse_xml_to_dict(child)
    if child.tag != 'object':
      result[child.tag] = child_result[child.tag]
    else:
      if child.tag not in result:
        result[child.tag] = []
      result[child.tag].append(child_result[child.tag])
  return {xml.tag: result}



In [22]:
def create_tf_example(data_info):
  # TODO START: Populate the following variables from your example.
  height = data_info['height'] # Image height
  width = data_info['width'] # Image width
  filename = data_info['filename'].encode() # Filename of the image. Empty if image is not from file
  encoded_image_data = data_info['encoded_image_data'] # Encoded image bytes
  image_format = data_info['image_format'].encode() # b'jpeg' or b'png'

  xmins = data_info['xmins'] # List of normalized left x coordinates in bounding box (1 per box)
  xmaxs = data_info['xmaxs'] # List of normalized right x coordinates in bounding box
             # (1 per box)
  ymins = data_info['ymins'] # List of normalized top y coordinates in bounding box (1 per box)
  ymaxs = data_info['ymaxs'] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
  classes_text = data_info['classes_text'] # List of string class name of bounding box (1 per box)
  classes = data_info['classes'] # List of integer class id of bounding box (1 per box)
  # TODO END
  tf_label_and_data = tf.train.Example(features=tf.train.Features(feature={
      'image/height': int64_feature(height),
      'image/width': int64_feature(width),
      'image/filename': bytes_feature(filename),
      'image/source_id': bytes_feature(filename),
      'image/encoded': bytes_feature(encoded_image_data),
      'image/format': bytes_feature(image_format),
      'image/object/bbox/xmin': float_list_feature(xmins),
      'image/object/bbox/xmax': float_list_feature(xmaxs),
      'image/object/bbox/ymin': float_list_feature(ymins),
      'image/object/bbox/ymax': float_list_feature(ymaxs),
      'image/object/class/text': bytes_list_feature(classes_text),
      'image/object/class/label': int64_list_feature(classes),
  }))
  return tf_label_and_data

In [23]:
def process_json_to_tf(json_path, output_path):
    food_bb_json = process_bb_box(json_path)
    assets = food_bb_json['assets']
    tags = list(map(lambda x: x['name'], food_bb_json['tags']))
    writer = tf.python_io.TFRecordWriter(output_path)
    for key in assets:
        image_metadata = assets[key]
        width = image_metadata['asset']['size']['width']
        height = image_metadata['asset']['size']['height']
        filename = image_metadata['asset']['path']
        image_format = image_metadata['asset']['format']
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []
        regions = image_metadata['regions']

        data_info = {
            'filename': filename,
            'image_format': image_format, 
            'width': width,
            'height': height
        }
        with tf.gfile.GFile(food_image_url + '/' + filename, 'rb') as fid:
            encoded_image_data = fid.read()
            data_info['encoded_image_data'] = encoded_image_data

        for bbox_region in regions:
            xmin = bbox_region['points'][0]['x'] / width
            ymin = bbox_region['points'][0]['y'] / height
            xmax = bbox_region['points'][2]['x'] / width
            ymax = bbox_region['points'][2]['y'] / height
            class_text = bbox_region['tags'][0]
            class_id = tags.index(class_text) + 1
            class_text = class_text.encode()
            xmins.append(xmin)
            ymins.append(ymin)
            xmaxs.append(xmax)
            ymaxs.append(ymax)
            classes_text.append(class_text)
            classes.append(class_id)
        data_info['xmins'] = xmins
        data_info['ymins'] = ymins
        data_info['xmaxs'] = xmaxs
        data_info['ymaxs'] = ymaxs
        data_info['classes_text'] = classes_text
        data_info['classes'] = classes
        tf_example = create_tf_example(data_info)
        writer.write(tf_example.SerializeToString())
    writer.close()

    

In [25]:
json_train_path = './vott-json-export/Food-Detection-App-export.json'
json_eval_path = './vott-json-export/Food-Detection-App-(Eval)-export.json'
tfrecord_train_path = './bbox-train.record'
tfrecord_eval_path = './bbox-eval.record'
process_json_to_tf(json_train_path, tfrecord_train_path)
process_json_to_tf(json_eval_path, tfrecord_eval_path)